In [10]:
import cv2
import numpy as np
from ultralytics import YOLO 
import serial, time
from keras.models import load_model

In [11]:
# Load your VGG16 waste classification model
model_classification = load_model('vgg16_biodegradable_classifier_model.h5')

In [12]:
# Initialize YOLO model
model_yolo = YOLO('yolov8n.pt')
# object classes
classNames = model_yolo.names

In [38]:
# Function for waste classification
def classify_waste(object_image):
    # Resize the object image to match your VGG16 input size
    resized_img = cv2.resize(object_image, (60, 60))
    # Expand dimensions to make it compatible with the model
    resized_img = np.expand_dims(resized_img, axis=0)
    # Perform classification
    prediction = model_classification.predict(resized_img)
    # Convert prediction to label
    label = np.argmax(prediction)
    # Map label to waste type
    if label == 0:
        return "biodegradable"
    else:
        return "non-biodegradable"

In [66]:

# Establish serial communication with Arduino
arduino = serial.Serial('COM3', 9600, timeout=1)  # Replace 'COM3' with the appropriate port

cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)

last_prediction = None
last_frame_sent_time = 0
frame_interval = 5  # in seconds

while True:
    current_time = time.time()
    
    # Send frame only if 5 seconds have passed since the last frame sent
    if current_time - last_frame_sent_time >= frame_interval:
        success, img = cap.read()
        if success:
            results = model_yolo(img, stream=True)

            for r in results:
                boxes = r.boxes

                for box in boxes:
                    x1, y1, x2, y2 = box.xyxy[0]
                    x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

                    cropped_object = img[int(y1):int(y2), int(x1):int(x2)]
                    waste_type = classify_waste(cropped_object)

                    if last_prediction != waste_type:
                        last_prediction = waste_type

                        # Send waste type to Arduino
                        if waste_type == "biodegradable":
                            arduino.write(b'B')  # Signal for biodegradable waste
                        else:
                            arduino.write(b'N')  # Signal for non-biodegradable waste

                        # Read response from Arduino
                        response = arduino.readline().decode().strip()
                        print("Arduino response:", response)

                    cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 255), 3)
                    cv2.putText(img, waste_type, (int(x1), int(y1) - 30), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

            cv2.imshow('Webcam', img)
            last_frame_sent_time = current_time

    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()



0: 480x640 (no detections), 179.9ms
Speed: 5.6ms preprocess, 179.9ms inference, 4.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 138.7ms
1/1 [==============================] - 0s 63ms/step
Arduino response: 
Speed: 0.0ms preprocess, 138.7ms inference, 6.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 134.6ms
1/1 [==============================] - 0s 70ms/step
Speed: 2.9ms preprocess, 134.6ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 135.0ms
1/1 [==============================] - 0s 69ms/step
Arduino response: 
Speed: 3.6ms preprocess, 135.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 130.2ms
Speed: 0.0ms preprocess, 130.2ms inference, 1.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 130.4ms
Speed: 3.0ms preprocess, 130.4ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 per